In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import

In [ ]:
import sys
from logging import INFO, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

In [ ]:
import os
import time
import math
import pathlib

import torch
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.cuda.amp import GradScaler

from src.utils.random_seed_helper import set_seeds, seed_worker, get_torch_generator
from src.frameworks.ddpm import DDPM, DDPMConfig
from src.datasets.dataset_lorenz96 import DatasetLorenz96
from src.configs.lorenz96_config import ExperimentLorenz96Config
from src.neural_networks.unet_1d import Unet1D
from src.training.loss_maker import make_loss
from src.training.optim_helper import optimize_ddpm

plt.rcParams["font.family"] = "serif"
plt.style.use("tableau-colorblind10")
os.environ["CUBLAS_WORKSPACE_CONFIG"] = r":4096:8"  # to make calculations deterministic

# Define constants

In [ ]:
DEVICE = "cuda:0"
ROOT_DIR = pathlib.Path(os.environ["PYTHONPATH"].split(":")[0]).resolve()
DL_DATA_FILE = str(ROOT_DIR / "data" / "DL_data" / "lorenz96" / "lorenz96_v00.nc")
DL_RESULT_DIR = str(ROOT_DIR / "data" / "DL_model" / "lonrenz96_v00")
os.makedirs(DL_RESULT_DIR, exist_ok=True)

# Make config

In [ ]:
config = ExperimentLorenz96Config(
    batch_size=100,
    loss_name="L2",
    learning_rate=1e-3,
    #
    n_features=32,
    list_channel=[1, 2, 4],
    #
    total_epochs=40,
    save_interval=4,
    use_auto_mix_precision=False,
    #
    ddpm=DDPMConfig(
        start_beta=1e-3,
        end_beta=3e1,
        n_timesteps=1_000,
        n_channels=32,
        n_spaces=32,  # dont change n_channels and n_spaces
    ),
)
config.save(f"{DL_RESULT_DIR}/config.yml")

In [ ]:
# You can load your saved config.
# config = ExperimentLorenz96Config.load(f"{DL_RESULT_DIR}/config.yml")
# config

# Make dataloader

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset=DatasetLorenz96(DL_DATA_FILE),
    batch_size=config.batch_size,
    drop_last=True,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    worker_init_fn=seed_worker,
    generator=get_torch_generator(),
)

In [ ]:
data = next(iter(dataloader))["y0"]
assert data.shape == (
    config.batch_size,
    config.ddpm.n_channels,
    config.ddpm.n_spaces,
)  # batch, time (=channel), space dims

## Plot

In [ ]:
plt.rcParams["font.size"] = 14
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[10, 4])

for i, ax in enumerate(axes.flatten()):
    d = dataloader.dataset.standardize_inversely(data[i].numpy())
    ts = np.arange(d.shape[0]) * 0.2  # dt = 0.2
    xs = np.linspace(0, 2 * math.pi, 32, endpoint=False)
    X, T = np.meshgrid(xs, ts, indexing="ij")
    ret = ax.pcolormesh(
        X, T, d.transpose(), vmin=-9, vmax=9, cmap="coolwarm", shading="nearest"
    )
    cbar = fig.colorbar(ret, ax=ax)
    ax.set_xlabel(r"Space, $x$")
    ax.set_ylabel(r"Time, $t$")
plt.tight_layout()
plt.show()

# Prepare for training

In [ ]:
set_seeds(42)

unet = Unet1D(
    dim=config.n_features,
    in_channels=32,  # num of times
    out_channels=32,
    padding_mode="circular",
    dim_mults=config.list_channel,
).to(DEVICE)

ddpm = DDPM(config=config.ddpm, neural_net=unet, device=DEVICE)

loss_fn = make_loss(loss_name=config.loss_name)
optimizer = torch.optim.AdamW(ddpm.parameters(), lr=config.learning_rate)
scaler = GradScaler()

# Train

In [ ]:
start_time = time.time()

all_scores: list[dict] = []
set_seeds(42)

with tqdm(total=config.total_epochs, desc="Training Progress", unit="step") as pbar:
    for _epoch in range(config.total_epochs):
        epoch = _epoch + 1

        loss = optimize_ddpm(
            dataloader=dataloader,
            ddpm=ddpm,
            loss_fn=loss_fn,
            optimizer=optimizer,
            epoch=epoch,
            mode="train",
            scaler=scaler,
            use_amp=config.use_auto_mix_precision,
        )
        all_scores.append({"epoch": epoch, "loss": loss})

        if epoch % config.save_interval == 0:
            p = f"{DL_RESULT_DIR}/model_weight_{epoch:06}.pth"
            torch.save(ddpm.net.state_dict(), p)

        if epoch % 10 == 0 or epoch == config.total_epochs:
            p = f"{DL_RESULT_DIR}/loss_history.csv"
            pd.DataFrame(all_scores).to_csv(p, index=False)

        pbar.set_postfix({"loss": loss})
        pbar.update(1)

end_time = time.time()
logger.info(f"Finished. Total elapsed time = {(end_time - start_time) / 60.} min")

# Test

In [ ]:
epoch = config.total_epochs
p = f"{DL_RESULT_DIR}/model_weight_{epoch:06}.pth"
unet.load_state_dict(torch.load(p, map_location=DEVICE, weights_only=False))
_ = unet.eval()

## Run sampling

In [ ]:
set_seeds(42)
dict_samples = ddpm.backward_sample_y(n_batches=10, n_return_steps=10)

## Plot generated samples

In [ ]:
last_samples = dict_samples[0]

plt.rcParams["font.size"] = 14
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[10, 4])

for i, ax in enumerate(axes.flatten()):
    d = last_samples[i].cpu().numpy()
    d = dataloader.dataset.standardize_inversely(d)
    ts = np.arange(d.shape[0]) * 0.2  # dt = 0.2
    xs = np.linspace(0, 2 * math.pi, 32, endpoint=False)
    X, T = np.meshgrid(xs, ts, indexing="ij")
    ret = ax.pcolormesh(
        X, T, d.transpose(), vmin=-9, vmax=9, cmap="coolwarm", shading="nearest"
    )
    cbar = fig.colorbar(ret, ax=ax)
    ax.set_xlabel(r"Space, $x$")
    ax.set_ylabel(r"Time, $t$")
plt.tight_layout()
plt.show()

## Plot intermediate states during generation

In [ ]:
plt.rcParams["font.size"] = 12
fig, axes = plt.subplots(2, 6, sharex=True, sharey=True, figsize=(15, 6))

for t, ax in zip(dict_samples.keys(), axes.flatten()):
    d = dict_samples[t][0].cpu().numpy()
    d = dataloader.dataset.standardize_inversely(d)

    ts = np.arange(d.shape[0]) * 0.2  # dt = 0.2
    xs = np.linspace(0, 2 * math.pi, 32, endpoint=False)
    X, T = np.meshgrid(xs, ts, indexing="ij")

    ret = ax.pcolormesh(
        X, T, d.transpose(), vmin=-9, vmax=9, cmap="coolwarm", shading="nearest"
    )

    cbar = fig.colorbar(ret, ax=ax)
    ax.set_xlabel(r"Space, $x$")
    ax.set_ylabel(r"Time, $t$")
    ax.set_title(f"Diffusion Step = {t}")
plt.tight_layout()
plt.show()